# 환경
- https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

In [1]:
!pip install mxnet-cu101
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==2.2.2
!pip install urllib3==1.25.10
# 실행 후 런타임 재시작 실행 (urllib3를 다시 로드 해야 함)

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-w42w9ujf
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-w42w9ujf
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=cc1d41e9acb0eb17468f361d8201a56052768bd0f678dfef4f92474ab889bcc8
  Stored in directory: /tmp/pip-ephem-wheel-cache-21p0wltm/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import json

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

In [4]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')
# data를 저장할 폴더 입니다. 환경에 맞게 수정 하세요.
data_dir = "/content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 네이버 영화 감정분류 (NSMC) (0.8968590153452686)
- https://github.com/e9t/nsmc/

In [ ]:
device = torch.device("cuda:0")

bert_model, bert_vocab = get_pytorch_kobert_model()

using cached model
using cached model


## Data

In [ ]:
!wget https://github.com/e9t/nsmc/raw/master/ratings_train.txt
!wget https://github.com/e9t/nsmc/raw/master/ratings_test.txt

--2020-10-27 13:00:27--  https://github.com/e9t/nsmc/raw/master/ratings_train.txt
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt [following]
--2020-10-27 13:00:27--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt.1’

ratings_train.txt.1 100%[===================>]  13.95M  69.2MB/s    in 0.2s    

2020-10-27 13:00:27 (69.2 MB/s) - ‘ratings_train.txt.1’ saved [14628807/14628807]

--2020-10-27 13:00:27--  https://github.com/e9t/nsmc/raw/master/

In [ ]:
train_data = pd.read_csv("ratings_train.txt", header=0, delimiter='\t', quoting=3)
print(f"전체 학습데이터의 개수: {len(train_data)}")
train_data.head(10)

전체 학습데이터의 개수: 150000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [ ]:
test_data = pd.read_csv("ratings_test.txt", header=0, delimiter='\t', quoting=3)
print(f"전체 테스트데이터의 개수: {len(test_data)}")
test_data.head(10)

전체 테스트데이터의 개수: 50000


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
5,7898805,"음악이 주가 된, 최고의 음악영화",1
6,6315043,진정한 쓰레기,0
7,6097171,"마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다",0
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...,0
9,6242223,"이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",1


In [ ]:
tokenizer = get_tokenizer()
bert_tokenizer = nlp.data.BERTSPTokenizer(tokenizer, bert_vocab, lower=False)

using cached model


In [ ]:
class NSMCDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = []
        self.labels = []
        for i, row in dataset.iterrows():
            if type(row["document"]) == str:
                self.sentences.append(transform([row["document"]]))
                self.labels.append(np.int32(row["label"]))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
data_train = NSMCDataset(train_data, bert_tokenizer, max_len, True, False)
data_test = NSMCDataset(test_data, bert_tokenizer, max_len, True, False)

In [ ]:
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

## Train

In [ ]:
class NSMCClassifier(nn.Module):
    def __init__(self, bert_model, d_model=768, n_output=2, dr_rate=0.1):
        super().__init__()
        self.bert_model = bert_model
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(d_model , n_output)
        self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert_model(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = NSMCClassifier(bert_model, dr_rate=0.1).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
warmup_ratio = 0.1
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
max_grad_norm = 1

best_acc = 0.0
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    p_bar = tqdm(train_dataloader)
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % 10 == 9:
            p_bar.update(10)
            p_bar.set_description(f'epoch {e+1} batch {batch_id+1} loss {loss.data.cpu().numpy():0.4f} acc {train_acc / (batch_id+1):0.4f}')
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    with torch.no_grad():
        model.eval()
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
    val_acc = test_acc
    if best_acc < val_acc:
        best_acc = val_acc
        save_path = os.path.join(data_dir, "kobert_nsmc.pth")
        torch.save(model.state_dict(), save_path)
        print(f"save model {save_path}")
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:146: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 1 train acc 0.8392678424378919



save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_nsmc.pth
epoch 1 test acc 0.8840543847137516


epoch 2 train acc 0.9070968864096357



save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_nsmc.pth
epoch 2 test acc 0.8915840792838875


...


save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_nsmc.pth
epoch 3 test acc 0.8948209718670077


epoch 4 train acc 0.9650071434240812



save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_nsmc.pth
epoch 4 test acc 0.8969988810741688


epoch 5 train acc 0.9779890145051194



epoch 5 test acc 0.8968590153452686


# Kakao KoNLI (0.6636687665782492)
- https://github.com/kakaobrain/KorNLUDatasets

In [5]:
device = torch.device("cuda:0")

bert_model, bert_vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


## Data

In [6]:
!wget https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/snli_1.0_train.ko.tsv
!wget https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv
!wget https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv

--2020-10-28 05:06:52--  https://github.com/kakaobrain/KorNLUDatasets/raw/master/KorNLI/snli_1.0_train.ko.tsv
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv [following]
--2020-10-28 05:06:53--  https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78486224 (75M) [text/plain]
Saving to: ‘snli_1.0_train.ko.tsv’

snli_1.0_train.ko.t 100%[===================>]  74.85M  70.9MB/s    in 1.1s    

2020-10-28 05:07:01 (70.9 MB/s) - ‘snli_1.0_train.ko.tsv’ sav

In [7]:
train_data = pd.read_csv('snli_1.0_train.ko.tsv', header=0, delimiter='\t', quoting=3)
print(f'전체 데이터의 개수: {len(train_data)}')
train_data.head(10)

전체 데이터의 개수: 550152


,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment
5,카메라에 웃고 손을 흔드는 아이들,아이들이 얼굴을 찌푸리고 있다,contradiction
6,한 소년이 빨간 다리 한가운데 스케이트보드에 뛰어오르고 있다.,소년은 인도를 따라 스케이트를 탄다.,contradiction
7,한 소년이 빨간 다리 한가운데 스케이트보드에 뛰어오르고 있다.,그 소년은 스케이트보드를 타는 묘기를 부린다.,entailment
8,한 소년이 빨간 다리 한가운데 스케이트보드에 뛰어오르고 있다.,소년이 안전 장비를 착용하고 있다.,neutral
9,나이 든 남자가 커피숍의 작은 테이블에 오렌지 주스를 들고 앉아 있고 밝은 색 셔츠...,나이 든 남자가 딸이 퇴근하기를 기다리면서 주스를 마신다.,neutral


In [8]:
dev_data = pd.read_csv('xnli.dev.ko.tsv', header=0, delimiter='\t', quoting=3)
print(f'전체 데이터의 개수: {len(dev_data)}')
dev_data.head(10)

전체 데이터의 개수: 2490


,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction
5,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 내가 무엇을 할 것인지 확신할 수 없어서 내가 보고를 하도록 배정받은 워싱턴으...,entailment
6,그는 갈 수 없었다.,그는 처음 초대를 받은 사람이고 그 경험을 즐겼다.,contradiction
7,그는 갈 수 없었다.,그는 참석하는 것을 허가받지 않았다.,entailment
8,그는 갈 수 없었다.,그는 박물관 개관식에 가는 것이 허용되지 않았다.,neutral
9,"그리고 저는 ""괜찮아"" 하고, 그게 다였어요!","내가 승낙한 후에, 그것은 끝났다.",entailment


In [9]:
test_data = pd.read_csv('xnli.test.ko.tsv', header=0, delimiter='\t', quoting=3)
print(f'전체 데이터의 개수: {len(test_data)}')
test_data.head(10)

전체 데이터의 개수: 5010


,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment
5,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",우리는 어떤 특권을 부여받기로 약속받았을지라도 똑같은 숫자를 받았다. 그것은 모두 ...,contradiction
6,"그들은 나에게 말했죠, 어, 끝에 있는 남자가 나를 만나자고 불러들일 거라고.",누구를 만난다는 얘기는 들은 적이 없어요.,contradiction
7,"그들은 나에게 말했죠, 어, 끝에 있는 남자가 나를 만나자고 불러들일 거라고.",한 남자를 나를 만나라고 부르겠다고 들었다.,entailment
8,"그들은 나에게 말했죠, 어, 끝에 있는 남자가 나를 만나자고 불러들일 거라고.",그 남자는 약간 늦게 나타났다.,neutral
9,네가 그거에 관해서 할 말이 너무 많아서 나는 그냥 넘어가겠어.,내가 그것에 대해 아는 모든 걸 말하고 싶어!,contradiction


In [10]:
tokenizer = get_tokenizer()
bert_tokenizer = nlp.data.BERTSPTokenizer(tokenizer, bert_vocab, lower=False)

using cached model


In [11]:
class KoNLIDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len, pad, pair):
        label_dict = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = []
        self.labels = []
        for i, row in dataset.iterrows():
            if type(row["sentence1"]) == str and type(row["sentence2"]) == str:
                self.sentences.append(transform([row["sentence1"], row["sentence2"]]))
                self.labels.append(np.int32(label_dict[row["gold_label"]]))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
max_len = 75
data_train = KoNLIDataset(train_data, bert_tokenizer, max_len, True, True)
data_dev = KoNLIDataset(dev_data, bert_tokenizer, max_len, True, True)
data_test = KoNLIDataset(test_data, bert_tokenizer, max_len, True, True)

In [13]:
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
dev_dataloader = torch.utils.data.DataLoader(data_dev, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

## Train

In [14]:
class NLIClassifier(nn.Module):
    def __init__(self, bert_model, d_model=768, n_output=3, dr_rate=0.1):
        super().__init__()
        self.bert_model = bert_model
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(d_model , n_output)
        self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert_model(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        out = self.dropout(pooler)
        return self.classifier(out)

In [15]:
model = NLIClassifier(bert_model, dr_rate=0.1).to(device)

In [16]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [17]:
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

In [18]:
num_epochs = 5
warmup_ratio = 0.1
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [19]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [21]:
max_grad_norm = 1

best_acc = 0.0
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    p_bar = tqdm(train_dataloader)
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % 10 == 9:
            p_bar.update(10)
            p_bar.set_description(f'epoch {e+1} batch {batch_id+1} loss {loss.data.cpu().numpy():0.4f} acc {train_acc / (batch_id+1):0.4f}')
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    with torch.no_grad():
        model.eval()
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(dev_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
    val_acc = test_acc
    if best_acc < val_acc:
        best_acc = val_acc
        save_path = os.path.join(data_dir, "kobert_konli.pth")
        torch.save(model.state_dict(), save_path)
        print(f"save model {save_path}")
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:146: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 1 train acc 0.7719008084215424
save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_konli.pth
epoch 1 test acc 0.6489693854995578


epoch 2 train acc 0.8513689368384321
save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_konli.pth
epoch 2 test acc 0.6636687665782492


epoch 3 train acc 0.8891873618704199
epoch 3 test acc 0.6558631741821397


epoch 4 train acc 0.9176638653018495
epoch 4 test acc 0.6613063660477453


epoch 5 train acc 0.9368765993951378
epoch 5 test acc 0.661223474801061


# KorQuAD (0.8033405172413793 / 0.7722701149425287)
- https://korquad.github.io/category/1.0_KOR.html

In [ ]:
device = torch.device("cuda:0")

bert_model, bert_vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


## Data

In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json

--2020-10-27 09:27:46--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   131MB/s    in 0.3s    

2020-10-27 09:27:47 (131 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2020-10-27 09:27:47--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [ ]:
with open('KorQuAD_v1.0_train.json') as f:
    train_data = json.load(f)
print(json.dumps(train_data["data"][0], indent=2, ensure_ascii=False))

{
  "paragraphs": [
    {
      "qas": [
        {
          "answers": [
            {
              "text": "교향곡",
              "answer_start": 54
            }
          ],
          "id": "6566495-0-0",
          "question": "바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?"
        },
        {
          "answers": [
            {
              "text": "1악장",
              "answer_start": 421
            }
          ],
          "id": "6566495-0-1",
          "question": "바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?"
        },
        {
          "answers": [
            {
              "text": "베토벤의 교향곡 9번",
              "answer_start": 194
            }
          ],
          "id": "6566495-0-2",
          "question": "바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?"
        },
        {
          "answers": [
            {
              "text": "파우스트",
              "answer_start": 15
            }
          ],
          "id": "6566518-0-0",
          "question": "1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?"
        },
        {
    

In [ ]:
with open('KorQuAD_v1.0_dev.json') as f:
    dev_data = json.load(f)
print(json.dumps(dev_data["data"][0], indent=2, ensure_ascii=False))

{
  "paragraphs": [
    {
      "qas": [
        {
          "answers": [
            {
              "text": "1989년 2월 15일",
              "answer_start": 0
            }
          ],
          "id": "6548850-0-0",
          "question": "임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?"
        },
        {
          "answers": [
            {
              "text": "임수경",
              "answer_start": 125
            }
          ],
          "id": "6548850-0-1",
          "question": "1989년 6월 30일 평양축전에 대표로 파견 된 인물은?"
        },
        {
          "answers": [
            {
              "text": "1989년",
              "answer_start": 0
            }
          ],
          "id": "6548853-0-0",
          "question": "임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?"
        },
        {
          "answers": [
            {
              "text": "학생회관 건물 계단",
              "answer_start": 365
            }
          ],
          "id": "6548853-0-1",
          "question": "임종석을 검거한 장소는 경희대 내 어디인가?"
        },

In [ ]:
tokenizer = get_tokenizer()
bert_tokenizer = nlp.data.BERTSPTokenizer(tokenizer, bert_vocab, lower=False)

using cached model


In [ ]:
tokens = bert_tokenizer('안녕하세요. 만나서 반갑습니다.')
token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)
tokens, token_ids

(['▁안', '녕', '하세요', '▁', '.', '▁만나', '서', '▁반', '갑', '습니다', '▁', '.'],
 [3135, 5724, 7814, 517, 54, 1933, 6553, 2207, 5345, 6701, 517, 54])

In [ ]:
def _is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False


def _tokenize_whitespace(string):
    word_tokens = []
    char_to_word = []
    prev_is_whitespace = True

    for c in string:
        if _is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                word_tokens.append(c)
            else:
                word_tokens[-1] += c
            prev_is_whitespace = False    
        char_to_word.append(len(word_tokens) - 1)
    
    return word_tokens, char_to_word


def _tokenize_vocab(bert_tokenizer, context_words):
    word_to_token = []
    context_tokens = []
    for (i, word) in enumerate(context_words):
        word_to_token.append(len(context_tokens))
        tokens = bert_tokenizer(word)
        for token in tokens:
            context_tokens.append(token)
    return context_tokens, word_to_token


def _improve_span(bert_tokenizer, context_tokens, token_start, token_end, char_answer):
    token_answer = " ".join(bert_tokenizer(char_answer))
    for new_start in range(token_start, token_end + 1):
        for new_end in range(token_end, new_start - 1, -1):
            text_span = " ".join(context_tokens[new_start : (new_end + 1)])
            if text_span == token_answer:
                return (new_start, new_end)
    return (token_start, token_end)


def dump_korquad(bert_tokenizer, json_data):
    dumps = []
    for data in tqdm(json_data["data"]):
        title = data["title"]
        for paragraph in data["paragraphs"]:
            context = paragraph["context"]
            context_words, char_to_word = _tokenize_whitespace(context)

            for qa in paragraph["qas"]:
                assert len(qa["answers"]) == 1
                qa_id = qa["id"]
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                answer_start = qa["answers"][0]["answer_start"]
                answer_end = answer_start + len(answer_text) - 1

                assert answer_text == context[answer_start:answer_end + 1]

                word_start = char_to_word[answer_start]
                word_end = char_to_word[answer_end]

                word_answer = " ".join(context_words[word_start:word_end + 1])
                char_answer = " ".join(answer_text.strip().split())
                assert char_answer in word_answer

                context_tokens, word_to_token = _tokenize_vocab(bert_tokenizer, context_words)

                token_start = word_to_token[word_start]
                if word_end < len(word_to_token) - 1:
                    token_end = word_to_token[word_end + 1] - 1
                else:
                    token_end = len(context_tokens) - 1

                token_start, token_end = _improve_span(bert_tokenizer, context_tokens, token_start, token_end, char_answer)

                data = {"qa_id": qa_id, "title": title, "question": bert_tokenizer(question), "context": context_tokens, "answer": char_answer, "token_start": token_start, "token_end":token_end}
                dumps.append(data)
    return dumps

In [ ]:
train_dumps = dump_korquad(bert_tokenizer, train_data)
len(train_dumps)

60407

In [ ]:
dev_dumps = dump_korquad(bert_tokenizer, dev_data)
len(dev_dumps)

5774

In [ ]:
class KorQuADDataset(Dataset):
    def __init__(self, dumps, bert_tokenizer, max_q_len, max_len):
        self.sentences = []
        self.labels = []
        for dump in dumps:
            token_start = dump['token_start']
            token_end = dump['token_end']
            question = dump['question'][:max_q_len]
            context_len = max_len - len(question) - 3
            context = dump['context'][:context_len]
            if token_end >= context_len:
                continue
            assert len(question) + len(context) <= max_len - 3
            
            tokens = ['[CLS]'] + question + ['[SEP]'] + context + ['[SEP]']
            token_len = len(tokens)
            segments = [0] * (len(question) + 2) + [1] * (len(context) + 1)
            tokens += ['[PAD]'] * (max_len - len(tokens))
            segments += [0] * (max_len - len(segments))
            assert len(tokens) == len(segments) == max_len
            token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)
            self.sentences.append((np.array(token_ids), token_len, np.array(segments)))

            token_start += (len(question) + 2)
            token_end += (len(question) + 2)
            self.labels.append((token_start, token_end))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
data_train = KorQuADDataset(train_dumps, bert_tokenizer, 64, 384)
data_dev = KorQuADDataset(dev_dumps, bert_tokenizer, 64, 384)

In [ ]:
batch_size = 16
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
dev_dataloader = torch.utils.data.DataLoader(data_dev, batch_size=batch_size)

## Train

In [ ]:
class KorQuadClassifier(nn.Module):
    def __init__(self, bert_model, d_model=768, dr_rate=0.1):
        super().__init__()
        self.bert_model = bert_model
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(d_model, 2)
        self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        logits, _ = self.bert_model(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        logits = self.dropout(logits)
        logits = self.classifier(logits)
        start_logits, end_logits = torch.split(logits, 1, dim=-1)
        return start_logits, end_logits

In [ ]:
model = KorQuadClassifier(bert_model, dr_rate=0.1).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
warmup_ratio = 0.1
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
max_grad_norm = 1

best_acc = 0.0
for e in range(num_epochs):
    train_acc = [0.0, 0.0]
    test_acc = [0.0, 0.0]
    model.train()
    p_bar = tqdm(train_dataloader)
    for batch_id, (token_ids, valid_length, segment_ids, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label_start = labels[0].long().to(device)
        label_end = labels[1].long().to(device)
        logits_start, logits_end = model(token_ids, valid_length, segment_ids)

        logits_start = torch.squeeze(logits_start, dim=-1)
        loss_start = loss_fn(logits_start, label_start)
        logits_end = torch.squeeze(logits_end, dim=-1)
        loss_end = loss_fn(logits_end, label_end)
        loss = loss_start + loss_end
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc[0] += calc_accuracy(logits_start, label_start)
        train_acc[1] += calc_accuracy(logits_end, label_end)

        if batch_id % 10 == 0:
            p_bar.update(10)
            p_bar.set_description(f'loss {loss_start.data.cpu().numpy():0.4f} {loss_end.data.cpu().numpy():0.4f} acc {train_acc[0] / (batch_id+1):0.4f} {train_acc[1] / (batch_id+1):0.4f}')
    with torch.no_grad():
        model.eval()
        for batch_id, (token_ids, valid_length, segment_ids, labels) in enumerate(dev_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label_start = labels[0].long().to(device)
            label_end = labels[1].long().to(device)
            logits_start, logits_end = model(token_ids, valid_length, segment_ids)
            test_acc[0] += calc_accuracy(logits_start, label_start)
            test_acc[1] += calc_accuracy(logits_end, label_end)
    val_acc = test_acc[0] + test_acc[1]
    if best_acc < val_acc:
        best_acc = val_acc
        save_path = os.path.join(data_dir, "kobert_korquad.pth")
        torch.save(model.state_dict(), save_path)
        print(f"save model {save_path}")
    print(f'epoch {e+1} test acc {test_acc[0] / (batch_id+1)}, {test_acc[1] / (batch_id+1)}')

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:146: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_korquad.pth
epoch 1 test acc 0.6566091954022989, 0.6570966748768473


save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_korquad.pth
epoch 2 test acc 0.7325790229885057, 0.7180316091954023


save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_korquad.pth
epoch 3 test acc 0.763418513957307, 0.7505387931034483


save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_korquad.pth
epoch 4 test acc 0.7843031609195402, 0.7566451149425287


save model /content/drive/My Drive/Colab Notebooks/lesson/nlp-hanwha/data/kobert_korquad.pth
epoch 5 test acc 0.8033405172413793, 0.7722701149425287
